In [9]:
import pandas as pd
import networkx as nw
import matplotlib.pyplot as plt 
import sys

In [10]:
df = pd.read_csv(r'C:\Users\KCC\Desktop\Flight_Data.csv')
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Pegasus Airlines,Sabiha Gökçen International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.898602,29.309200,312,Tehran,Iran,35.416100,51.152199,3305,1998.541333,2.982660,969.270667
1,Turkish Airlines,Sabiha Gökçen International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.898602,29.309200,312,Tehran,Iran,35.416100,51.152199,3305,1998.541333,2.982660,969.270667
2,Turkish Airlines,Atatürk International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.976898,28.814600,163,Tehran,Iran,35.416100,51.152199,3305,2040.978811,3.035377,990.489405
3,Emirates,Dubai International Airport,Imam Khomeini International Airport,Dubai,United Arab Emirates,25.252800,55.364399,62,Tehran,Iran,35.416100,51.152199,3305,1199.863567,1.990514,569.931784
4,Etihad Airways,Abu Dhabi International Airport,Imam Khomeini International Airport,Abu Dhabi,United Arab Emirates,24.433001,54.651100,88,Tehran,Iran,35.416100,51.152199,3305,1266.681453,2.073517,603.340726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12845,Kenmore Air,Boeing Field King County International Airport,William R Fairchild International Airport,Seattle,United States,47.529999,-122.302002,21,Port Angeles,United States,48.120201,-123.500000,291,110.930397,0.637802,25.465199
12846,Kenmore Air,Boeing Field King County International Airport,Orcas Island Airport,Seattle,United States,47.529999,-122.302002,21,Eastsound,United States,48.708199,-122.910004,31,138.564899,0.672130,39.282450
12847,Kenmore Air,William R Fairchild International Airport,Boeing Field King County International Airport,Port Angeles,United States,48.120201,-123.500000,291,Seattle,United States,47.529999,-122.302002,21,110.930397,0.637802,25.465199
12848,Kenmore Air,Friday Harbor Airport,Boeing Field King County International Airport,Friday Harbor,United States,48.521999,-123.024002,113,Seattle,United States,47.529999,-122.302002,21,122.677708,0.652395,31.338854


In [11]:
G = nw.DiGraph()

In [12]:
G = nw.from_pandas_edgelist(df, source = 'SourceAirport', target = 'DestinationAirport', edge_attr = 'Distance')

In [13]:
G.nodes()

NodeView(('Sabiha Gökçen International Airport', 'Imam Khomeini International Airport', 'Atatürk International Airport', 'Dubai International Airport', 'Abu Dhabi International Airport', 'Sharjah International Airport', 'Hamad International Airport', 'Esenboğa International Airport', 'Adnan Menderes International Airport', "Chicago O'Hare International Airport", 'Los Angeles International Airport', 'John F Kennedy International Airport', 'Antalya International Airport', 'Adana Airport', 'Milas Bodrum International Airport', 'Diyarbakir Airport', 'Dalaman International Airport', 'Erzurum International Airport', 'Elazığ Airport', 'Şanlıurfa GAP Airport', 'Gaziantep International Airport', 'Malatya Tulga Airport', 'Trabzon International Airport', 'Lester B. Pearson International Airport', 'Washington Dulles International Airport', 'George Bush Intercontinental Houston Airport', 'Esfahan Shahid Beheshti International Airport', 'Shahid Ashrafi Esfahani Airport', 'Mashhad International Airpo